# download the dataset from here:
#https://www.microsoft.com/en-us/research/project/mslr/

### libSVM format
read about this file format here: https://catboost.ai/docs/concepts/input-data_libsvm.html

In [3]:
import os
import pickle as pkl
import numpy as np
import pandas as pd
from sklearn.datasets import load_svmlight_file
import xgboost as xgb
from xgboost.testing.data import RelDataCV
from sklearn.metrics import ndcg_score, average_precision_score

In [4]:
def load_mlsr_10k(data_path: str, cache_path: str) -> RelDataCV:
    """Load the MSLR10k dataset from data_path and cache a pickle object in cache_path.

    Returns
    -------

    A list of tuples [(X, y, qid), ...].

    """
    root_path = os.path.expanduser('C:\MSLR-WEB10K')
    cacheroot_path = os.path.expanduser('C:\MSLR-WEB10K-cache')
    cache_path = os.path.join(cacheroot_path, 'C:\MSLR_10K_LETOR.pkl')

    # Use only the Fold1 for demo:
    # Train,      Valid, Test
    # {S1,S2,S3}, S4,    S5
    fold = 1

    if not os.path.exists(cache_path):
        fold_path = os.path.join(root_path, f"Fold{fold}")
        train_path = os.path.join(fold_path, "train.txt")
        valid_path = os.path.join(fold_path, "vali.txt")
        test_path = os.path.join(fold_path, "test.txt")
        X_train, y_train, qid_train = load_svmlight_file(
            train_path, query_id=True, dtype=np.float32
        )
        y_train = y_train.astype(np.int32)
        qid_train = qid_train.astype(np.int32)

        X_valid, y_valid, qid_valid = load_svmlight_file(
            valid_path, query_id=True, dtype=np.float32
        )
        y_valid = y_valid.astype(np.int32)
        qid_valid = qid_valid.astype(np.int32)

        X_test, y_test, qid_test = load_svmlight_file(
            test_path, query_id=True, dtype=np.float32
        )
        y_test = y_test.astype(np.int32)
        qid_test = qid_test.astype(np.int32)

        data = RelDataCV(
            train=(X_train, y_train, qid_train),
            test=(X_test, y_test, qid_test),
            max_rel=4,
        )

        with open(cache_path, "wb") as fd:
            pkl.dump(data, fd)

    with open(cache_path, "rb") as fd:
        data = pkl.load(fd)

    return data

In [5]:
def ranking_demo():
    """Demonstration for learning to rank with relevance degree."""
    data = load_mlsr_10k('C:\MSLR-WEB10K', 'C:\MSLR-WEB10K-cache')

    # Sort data according to query index
    X_train, y_train, qid_train = data.train
    sorted_idx = np.argsort(qid_train)
    X_train = X_train[sorted_idx]
    y_train = y_train[sorted_idx]
    qid_train = qid_train[sorted_idx]

    X_test, y_test, qid_test = data.test
    sorted_idx = np.argsort(qid_test)
    X_test = X_test[sorted_idx]
    y_test = y_test[sorted_idx]
    qid_test = qid_test[sorted_idx]

    ranker = xgb.XGBRanker(
        tree_method="hist",
        device="cuda",
        lambdarank_pair_method="mean",
        lambdarank_num_pair_per_sample=5,
        eval_metric=["ndcg@1", "ndcg@8"],
    )
    ranker.fit(
        X_train,
        y_train,
        qid=qid_train,
        eval_set=[(X_test, y_test)],
        eval_qid=[qid_test],
        verbose=True,
    )
    # Save the model
    model_IR = "ranker_model.json"
    ranker.save_model(model_IR)

    return ranker, X_train, y_train, qid_train, X_test, y_test, qid_test, model_IR 

ranker, X_train, y_train, qid_train, X_test, y_test, qid_test, model_IR = ranking_demo()


C:\Users\dell\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:15:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


[0]	validation_0-ndcg@1:0.37069	validation_0-ndcg@8:0.39967
[1]	validation_0-ndcg@1:0.41020	validation_0-ndcg@8:0.42229
[2]	validation_0-ndcg@1:0.41790	validation_0-ndcg@8:0.43076
[3]	validation_0-ndcg@1:0.43181	validation_0-ndcg@8:0.43708
[4]	validation_0-ndcg@1:0.43819	validation_0-ndcg@8:0.44078
[5]	validation_0-ndcg@1:0.44350	validation_0-ndcg@8:0.44393
[6]	validation_0-ndcg@1:0.44400	validation_0-ndcg@8:0.44659
[7]	validation_0-ndcg@1:0.44378	validation_0-ndcg@8:0.44911
[8]	validation_0-ndcg@1:0.44407	validation_0-ndcg@8:0.45207
[9]	validation_0-ndcg@1:0.44843	validation_0-ndcg@8:0.45476
[10]	validation_0-ndcg@1:0.44948	validation_0-ndcg@8:0.45637
[11]	validation_0-ndcg@1:0.45174	validation_0-ndcg@8:0.45920
[12]	validation_0-ndcg@1:0.45643	validation_0-ndcg@8:0.46284
[13]	validation_0-ndcg@1:0.45774	validation_0-ndcg@8:0.46476
[14]	validation_0-ndcg@1:0.45888	validation_0-ndcg@8:0.46543
[15]	validation_0-ndcg@1:0.45863	validation_0-ndcg@8:0.46742
[16]	validation_0-ndcg@1:0.46047	v

### Exercises
1) save the generated model into disk
2) load the generated model from disk
3) use the `predict` method and try it on a number of samples
4) compare the performance of a numer of methods: rank:ndcg (default), rank:map, and rank:pairwise
   https://xgboost.readthedocs.io/en/stable/parameter.html#learning-task-parameters

In [6]:
#load the model
loaded_ranker = xgb.XGBRanker()
loaded_ranker.load_model(model_IR)

In [7]:
n_samples = 8 
sample_test_data = X_test[:n_samples]
predictions = loaded_ranker.predict(sample_test_data)
print("Predictions:", predictions)

Predictions: [-0.49255788 -0.02598316 -0.55691683  0.22683729  0.26758316  1.0661651
  0.31244534  0.45037895]


In [8]:
import numpy as np

def to_binary_labels(y, threshold=None):
    if threshold is not None:
        return np.where(y > threshold, 1, 0)
    else:
        return y 

def train_and_evaluate(objective, X_train, y_train, qid_train, X_test, y_test, qid_test, threshold=None):
    # Convert labels
    y_train_bin = to_binary_labels(y_train, threshold)
    y_test_bin = to_binary_labels(y_test, threshold)

    ranker = xgb.XGBRanker(
        tree_method="hist",
        objective=objective,
        eval_metric=["ndcg@1", "ndcg@8"],
    )
    ranker.fit(
        X_train,
        y_train_bin,
        qid=qid_train,
        eval_set=[(X_test, y_test_bin)],
        eval_qid=[qid_test],
        verbose=True,
    )
    predictions = ranker.predict(X_test)
    score = ndcg_score([y_test_bin], [predictions])

    return score

threshold = 2
ndcg_score_ndcg = train_and_evaluate("rank:ndcg", X_train, y_train, qid_train, X_test, y_test, qid_test)
ndcg_score_map = train_and_evaluate("rank:map", X_train, y_train, qid_train, X_test, y_test, qid_test, threshold)
ndcg_score_pairwise = train_and_evaluate("rank:pairwise", X_train, y_train, qid_train, X_test, y_test, qid_test)

#performances
print("rank:ndcg score:", ndcg_score_ndcg)
print("rank:map score:", ndcg_score_map)
print("rank:pairwise score:", ndcg_score_pairwise)


[0]	validation_0-ndcg@1:0.38987	validation_0-ndcg@8:0.40893
[1]	validation_0-ndcg@1:0.42807	validation_0-ndcg@8:0.43250
[2]	validation_0-ndcg@1:0.43772	validation_0-ndcg@8:0.43896
[3]	validation_0-ndcg@1:0.43785	validation_0-ndcg@8:0.44395
[4]	validation_0-ndcg@1:0.44232	validation_0-ndcg@8:0.44802
[5]	validation_0-ndcg@1:0.44810	validation_0-ndcg@8:0.44963
[6]	validation_0-ndcg@1:0.45144	validation_0-ndcg@8:0.45352
[7]	validation_0-ndcg@1:0.44869	validation_0-ndcg@8:0.45500
[8]	validation_0-ndcg@1:0.45181	validation_0-ndcg@8:0.45665
[9]	validation_0-ndcg@1:0.45338	validation_0-ndcg@8:0.45892
[10]	validation_0-ndcg@1:0.45278	validation_0-ndcg@8:0.46004
[11]	validation_0-ndcg@1:0.45642	validation_0-ndcg@8:0.46283
[12]	validation_0-ndcg@1:0.46048	validation_0-ndcg@8:0.46618
[13]	validation_0-ndcg@1:0.46480	validation_0-ndcg@8:0.46843
[14]	validation_0-ndcg@1:0.46542	validation_0-ndcg@8:0.46962
[15]	validation_0-ndcg@1:0.46223	validation_0-ndcg@8:0.47151
[16]	validation_0-ndcg@1:0.46688	v